# Current results of RABS_SIMPLIFY - 27/07/2022

In [1]:
import numpy as np
import geopandas as gpd
from shapely.ops import polygonize

import folium

In [2]:
from rabs_simplify import roundabout_simpl

/Users/gregoriomaya/mambaforge/envs/test/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lading data

In [3]:
#loading data from file
mad = gpd.read_file('/Users/gregoriomaya/Desktop/GSoc_2022/gsoc2022_network_simpl/data/madrid2062.gpkg')
mad.set_index(['u', 'v', 'key'], inplace=True)

#creating polygons for the function
mad_polys = polygonize(mad.geometry)
mad_polys = gpd.GeoDataFrame(geometry = [g for g in mad_polys], crs= mad.crs)

mad.head(3)

osmid  oneway lanes                name  \
u      v          key                                                
171946 26513145   0    807334397    True     4  Calle de Velázquez   
       3280496563 0    807334397    True     4  Calle de Velázquez   
       1209330272 0    321307751    True   nan  Calle de Velázquez   

                           highway maxspeed reversed  length        from  \
u      v          key                                                      
171946 26513145   0       tertiary       50    False  41.093      171946   
       3280496563 0       tertiary       50    False  30.852  3280496563   
       1209330272 0    residential      nan    False  32.461  1209330272   

                             to junction bridge  ref access tunnel width  \
u      v          key                                                      
171946 26513145   0    26513145     None    nan  nan    nan    nan  None   
       3280496563 0      171946     None    nan  nan    nan    nan  None   
       1209330272 0      171946     None    nan  nan    nan    nan  None   

                      landuse est_width  \
u      v          key                     
171946 26513145   0      None      None   
       3280496563 0      None      None   
       1209330272 0      None      None   

                                                                geometry  
u      v          key                                                     
171946 26513145   0    LINESTRING (600247.796 646721.109, 600250.422 ...  
       3280496563 0    LINESTRING (600242.659 646690.974, 600244.482 ...  
       1209330272 0    LINESTRING (600265.192 646697.507, 600261.267 ...

## Running the function and displaying results

In [4]:
simpl_mad = roundabout_simpl(mad, mad_polys)
simpl_mad.tail(3)

/Users/gregoriomaya/mambaforge/envs/test/lib/python3.10/site-packages/pandas/core/dtypes/inference.py:383: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  iter(obj)  # Can iterate over it.
/Users/gregoriomaya/mambaforge/envs/test/lib/python3.10/site-packages/pandas/core/dtypes/inference.py:384: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  len(obj)  # Has a length associated with it.
/Users/gregoriomaya/mambaforge/envs/test/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts o

,,,osmid,oneway,lanes,name,highway,maxspeed,reversed,length,from,to,junction,bridge,ref,access,tunnel,width,landuse,est_width,geometry
u,v,key,,,,,,,,,,,,,,,,,,,
1156937193,1156937473,0,100078448,True,1,Calle Batalla de Tesalia,residential,30,False,199.148,1156937193,1156937473,None,nan,nan,nan,nan,None,None,None,"LINESTRING (609201.227 645452.252, 609205.567 ..."
311839589,311839831,0,"[251072992, 663204503]",False,nan,Avenida de Fermina Sevillano,residential,nan,True,127.313,311839831,311839589,None,nan,nan,nan,nan,None,None,None,"LINESTRING (609896.321 649204.148, 609911.496 ..."
1492770542,311837857,0,71380724,False,nan,Avenida de Fermina Sevillano,residential,nan,False,14.087,311837857,1492770542,None,nan,nan,nan,nan,None,None,None,"LINESTRING (610067.969 649368.279, 610080.673 ..."


In [ ]:
m = mad.explore(tiles = "CartoDB positron", color =  'black', name = 'original network' )
simpl_mad.explore(m=m, color = 'red', name = 'simplifyed (roundabouts) output')

folium.LayerControl().add_to(m)
m

## testing time


In [6]:
import rabs_simplify

In [7]:
removed_edges = len(mad) - len(simpl_mad)

print('Size of the original GDF : ',len(mad) )
print('Size of the NEW original GDF : ',len(simpl_mad), -round(removed_edges/len(mad)*100,1),'%' )

rabs = rabs_simplify.selecting_rabs_from_poly(mad_polys)
rabs_modified = len(rabs.index_right.unique())                              

print('Total number of ROUNDABOUTS modified : ', rabs_modified)

Size of the original GDF :  47381
Size of the NEW original GDF :  46124 -2.7 %
Total number of ROUNDABOUTS modified :  689


In [8]:
%%timeit
roundabout_simpl(mad, mad_polys)

/Users/gregoriomaya/mambaforge/envs/test/lib/python3.10/site-packages/pandas/core/dtypes/inference.py:383: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  iter(obj)  # Can iterate over it.
/Users/gregoriomaya/mambaforge/envs/test/lib/python3.10/site-packages/pandas/core/dtypes/inference.py:384: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  len(obj)  # Has a length associated with it.
/Users/gregoriomaya/mambaforge/envs/test/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts o

9.25 s ± 801 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
